<a href="https://colab.research.google.com/github/eyupdalan/BLM6114-project/blob/main/BLM6114_project_tr_llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Kütüphaneler

In [1]:
!pip install -q torch accelerate transformers datasets huggingface-hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.8 MB/s eta 0:00:00


## Modellere göre sınıflandırma (Çözebilir mi?)

Modeller:

https://huggingface.co/ytu-ce-cosmos/Turkish-Llama-8b-DPO-v0.1



### Modelin yüklenmesi

In [2]:
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from huggingface_hub import notebook_login

# pipeline'ları oluşturma
turkish_llama = pipeline(
    "text-generation",
    model="ytu-ce-cosmos/Turkish-Llama-8b-DPO-v0.1",
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0


### Değerlendirme Fonksiyonu

In [13]:
import re

def extract_number(text):
    # Metindeki sayıları bul, sonuncusunu float’a çevir. Son sayı cevap.
    nums = re.findall(r"[-+]?\d+\.?\d*", text.replace(",","."))
    return float(nums[-1]) if nums else None

def predict_tr_llama(question):
  prompt = (
    f"Soru: {question}\n"
    "Soruya kısa ve net bir yanıt ver, adım adım çözme, yanıtlarında ‘24 \times \frac’ gibi notasyonlar kullanma, bunu da ‘Sonuç: <değer>’ formatında göster."
  )

  messages = [
    {"role": "user", "content": prompt},
  ]
  terminators = [
    turkish_llama.tokenizer.eos_token_id,
    turkish_llama.tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = turkish_llama(
    messages,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
  )

  assistant_response = outputs[0]["generated_text"][-1]["content"].strip()
  return assistant_response

def label_prediction(question, true_answer):
  tr_llama_result = predict_tr_llama(question)
  print("---------------------------------------")
  print(f"tr_llama_result: {tr_llama_result}")
  print("---------------------------------------")

  tr_llama_prediction = extract_number(tr_llama_result)
  print("---------------------------------------")
  print(f"tr_llama_prediction: {tr_llama_prediction}")
  print("---------------------------------------")

  real = extract_number(true_answer)
  # Tahmin gerçek değerle eşleşiyorsa 1, yoksa 0
  is_tr_llama_correct = 1 if (tr_llama_prediction is not None and real is not None and abs(tr_llama_prediction-real)<1e-3) else 0
  return tr_llama_result, is_tr_llama_correct

### Örnek Soru-Cevap Karşılaştırması

In [15]:
import os
os.environ["TRANSFORMERS_VERBOSITY"] = "info"

df = pd.read_csv("gsm8k_tr_classified.csv")

results = []

# Sadece ilk 10 kayıt için döngü
for i in range(10):
  q = df.question.iloc[i]
  true_ans = df.answer.iloc[i]

  tr_llama_result, is_tr_llama_correct = label_prediction(q, true_ans)

  # DataFrame'e yeni sütunlar ekleyelim
  df.loc[i, "tr_llama_result"] = tr_llama_result
  df.loc[i, "is_tr_llama_correct"] = is_tr_llama_correct

# Sonuçları DataFrame olarak görüntüle (İlk 10 satır için)
print(df[:10])

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


---------------------------------------
tr_llama_result: Soruyu adım adım çözelim:

1. Borris Tekel Bayisi her 6 ayda bir 90 kg üzüm kullanıyor. Bir yılda 2 kez üzüm kullanıyor (2 kez 6 ay = 1 yıl).
2. Üretimini yüzde 20 oranında artırmayı düşünüyor. Bu, 90 kg üzümün %20 artması anlamına gelir. Yani 90 kg * 0,20 = 18 kg üzüm daha kullanacak.
3. Bir yılda 2 kez üzüm kullanacağı için bu 18 kg üzümü 2 kez kullanacak.
4. Toplamda 90 kg + 18 kg = 108 kg üzüm kullanacak.
5. Bir yılda 2 kez üzüm kullanacağı için 108 kg üzüm / 2 = 54 kg üzüm kullanacak.

Sonuç: Borris Tekel Bayisi, üretimini artırdıktan sonra bir yıl içinde 54 kez 54 kg üzüme ihtiyacı olacak.
---------------------------------------
---------------------------------------
tr_llama_prediction: 54.0
---------------------------------------


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


---------------------------------------
tr_llama_result: Soruyu adım adım çözelim:

1. Katherine iki düzine yaşına geldiğinde kaç yaşında olduğunu bulalım.
   Bir düzine 10, iki düzine 20'dir. Yani Katherine 20 yaşındadır.
   
2. Mel, Katherine'den üç yaş küçüktür. Bu nedenle Mel'in yaşı 20 - 3 = 17'dir.
   
Sonuç: Mel 17 yaşında olacak.
---------------------------------------
---------------------------------------
tr_llama_prediction: 17.0
---------------------------------------


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

---------------------------------------
tr_llama_result: Sorunu adım adım çözelim:

1. Düz kumda hızı: 60 mil/saat
2. Yokuş aşağı hızı: 60 + 12 = 72 mil/saat (düz kumda hıza göre 12 mil daha hızlı)
3. Yokuş yukarı hızı: 60 - 18 = 42 mil/saat (düz kumda hıza göre 18 mil daha yavaş)

Conner'ın arabayı üçte bir oranında düz kumda, üçte bir oranında yokuş yukarı ve üçte bir oranında yokuş aşağı sürdüğünü biliyoruz. Yani:

- Düz kumda: 1/3 * 60 = 20 mil/saat
- Yokuş yukarı: 1/3 * 42 = 14 mil/saat
- Yokuş aşağı: 1/3 * 72 = 24 mil/saat

Ortalama hızı bulmak için bu hızları toplayıp 3'e bölüyoruz:

(20 + 14 + 24) / 3 = 58 mil/saat

Sonuç: 58
---------------------------------------
---------------------------------------
tr_llama_prediction: 58.0
---------------------------------------


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


---------------------------------------
tr_llama_result: Sonuç: 4000 gram

Adım adım çözümü:

1. Mağaza her hafta 20 paket şeker satıyor.
2. Her pakette 100 gram şeker bulunuyor.
3. Bir haftada satılan toplam şeker miktarını bulmak için, 20 paket * 100 gram/paket = 2000 gram.
4. Haftanın her gününü dikkate alarak, bir haftada satılan toplam şeker miktarını bulmak için, 2000 gram * 1 hafta = 4000 gram.
---------------------------------------
---------------------------------------
tr_llama_prediction: 4000.0
---------------------------------------


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


---------------------------------------
tr_llama_result: Sonuç: 4.500$

Adım adım çözüm:

1. James 20.000 $ değerindeki arabasını %80'ine sattı:
20.000 $ * 0,8 = 16.000 $

2. 16.000 $ aldıktan sonra 30.000 $ değerindeki araba için %90'a indirim aldı:
30.000 $ * 0,9 = 27.000 $

3. Cebinden çıkan tutar:
27.000 $ - 16.000 $ = 11.000 $

4. Ancak, 11.000 $ sadece yeni araba için ödediği tutarı gösteriyor. James 16.000 $ almıştı, dolayısıyla cebinden çıkan net tutar:
16.000 $ - 11.000 $ = 4.500 $

Sonuç: 4.500$
---------------------------------------
---------------------------------------
tr_llama_prediction: 4.5
---------------------------------------
---------------------------------------
tr_llama_result: Sonuç: 5,30 $

Adım adım hesaplamalar:

1. Hercai menekşe: 5 adet × 2,50 $ = 12,50 $
2. Ortanca: 1 adet × 12,50 $ = 12,50 $
3. Petunyalar: 5 adet × 1,00 $ = 5,00 $

Toplam fiyat: 12,50 $ + 12,50 $ + 5,00 $ = 30,00 $

İndirim: 30,00 $ × 10% = 3,00 $

İndirimli fiyat: 30,00 $ - 3,00 $ = 2